In [1]:
import pandas as pd
import os

In [2]:
data = pd.read_csv("../../../nature_cancer_dataset/single_cell_data.csv")

In [3]:
data.head()

,file_id,metabricId,core_id,ImageNumber,ObjectNumber,HH3_total,CK19,CK8_18,Twist,CD68,...,DNA1,DNA2,H3K27me3,CK5,Fibronectin,Location_Center_X,Location_Center_Y,SOM_nodes,pg_cluster,description
0,MB0000_1_527,MB-0000,1,527,1,1.058000,0.000000,0.000000,0.000000,0.618000,...,17.330166,17.718166,0.903333,0.344000,35.934502,161.83333,6.000000,6,24,Fibroblasts
1,MB0000_1_527,MB-0000,1,527,2,0.889243,0.166252,0.166487,0.109148,0.071443,...,28.806173,52.767433,0.781896,0.111391,16.294783,177.30435,15.539130,6,24,Fibroblasts
2,MB0000_1_527,MB-0000,1,527,3,0.861557,0.150873,0.099861,0.064506,0.025316,...,17.687202,29.113506,0.872304,0.542215,16.348177,293.51898,19.886076,6,24,Fibroblasts
3,MB0000_1_527,MB-0000,1,527,4,1.746239,0.069159,0.043478,0.021739,0.623623,...,29.021986,54.447716,0.664442,0.172638,9.053551,165.04347,22.210144,4,20,Fibroblasts CD68+
4,MB0000_1_527,MB-0000,1,527,5,0.979274,0.144429,0.125500,0.067714,0.028940,...,21.988190,43.471428,0.446310,0.057833,9.991821,108.88095,28.523809,6,24,Fibroblasts


In [18]:
data_clinical = pd.read_table("../../../nature_cancer_dataset/data_clinical.txt").drop([0, 1, 2, 3, 4])
TNBC_clinical = data_clinical[(data_clinical['PR Status'] == '-') & (data_clinical['ER Status'] == '-') & (data_clinical['HER2 Status'] == '-')]

In [19]:
tnbc_identifier = TNBC_clinical['#Patient Identifier']
TNBC_data = data[data.metabricId.isin(tnbc_identifier)]

In [20]:
TNBC_data['description'].unique()

array(['Fibroblasts', 'Myofibroblasts', 'Endothelial', 'T cells',
       'Fibroblasts CD68+', 'B cells', 'HER2+', 'Vascular SMA+',
       'HR+ CK7-', 'Myoepithelial', 'HR- CKlow CK5+', 'HR- CK7-',
       'HR- CK7+', 'Hypoxia', 'HR- Ki67+', 'Basal CKlow',
       'Macrophages Vim+ CD45low', 'HRlow CKlow', 'HR+ CK7- Slug+',
       'HR+ CK7- Ki67+', 'Macrophages Vim+ Slug+',
       'Macrophages Vim+ Slug-'], dtype=object)

In [21]:
def cell_type_converter(t):
    d = {'Fibroblasts': 'Fibroblasts', 
         'Myofibroblasts': 'Fibroblasts', 
         'Endothelial': 'Endothelial', 
         'T cells': 'T cells',
         'Fibroblasts CD68+': 'Fibroblasts', 
         'B cells': 'B cells', 
         'HER2+': 'Cancer', 
         'Vascular SMA+': 'Vascular SMA+',
         'HR+ CK7-': 'Cancer',
         'Myoepithelial':'Myoepithelial',
         'HR- CKlow CK5+': 'Cancer',
         'HR- CK7-': 'Cancer',
         'HR- CK7+': 'Cancer',
         'Hypoxia': 'Cancer',
         'HR- Ki67+': 'Cancer',
         'Basal CKlow': 'Cancer',
         'Macrophages Vim+ CD45low': 'Macrophages',
         'HRlow CKlow': 'Cancer', 
         'HR+ CK7- Slug+': 'Cancer',
         'HR+ CK7- Ki67+': 'Cancer', 
         'Macrophages Vim+ Slug+': 'Macrophages',
         'Macrophages Vim+ Slug-': 'Macrophages'}
    return d[t]

TNBC_data['description'] = TNBC_data['description'].apply(cell_type_converter)

/home/fabio.lipreri/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [25]:
TNBC_data['description'].unique()

array(['Fibroblasts', 'Endothelial', 'T cells', 'B cells', 'Cancer',
       'Vascular SMA+', 'Myoepithelial', 'Macrophages'], dtype=object)

In [23]:
for i, x in TNBC_data.groupby('ImageNumber'):
    p = os.path.join("../../../output/tnbc_nature_cancer_dataset/patient{}_cell_positions.csv".format(i))
    x = x[['Location_Center_X', 'Location_Center_Y', 'description', 'ObjectNumber']]
    x.columns = ['x', 'y', 'cell_type', 'label']
    x['size_x'] = int(x['x'].max())
    x['size_y'] = int(x['y'].max())
    x.to_csv(p, header=True, index=False, sep=',')

/home/fabio.lipreri/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/fabio.lipreri/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
image_id = data['ImageNumber'].unique()
for i in image_id:
    cells = data[data['ImageNumber']]